In [14]:
import os
import torch
import torch.optim as optim
import torch.utils.data as Dataset

# from Load import load_shot, load_eig
from FAUSTDataset import FAUSTDataset
from ResNet import DFM
from EuclideanLoss import EuclideanLoss

In [15]:
data_dir = '../data/'
train_reg_shot = 'training/reg_shot/'
train_scan_shot = 'training/scan_shot/'
train_eigen = 'training/reg_lb/'
train_dist = 'training/l2_dist/'
test_scan_shot = 'test/scan_shot/'
test_eigen = 'test/scan_lb/'
inter = 'inter_challenge.txt'
intra = 'intra_challenge.txt'

In [16]:
tr=True
batch_size=4
lr = 0.001
max_epoch=50
eignum = 120
num_layer=7

In [18]:
# load training data
# shot = []
# eigen = []
# dist = []
# if tr:
#     streig = "%d" % (eignum)
#     for i in range(100):
#         strnum = '%03d' % (i)
# #                 data_dir, train_reg_shot, 
#         fn = ''.join([data_dir, train_reg_shot, 'tr_reg_res_', strnum, '.txt'])
#         shot.append(load_shot(fn))
#         fn_eig = ''.join([data_dir, train_eigen, 'tr_reg_', streig, '_', strnum, '.h5'])
#         eigen.append(load_eig(fn_eig))
# #         fn_dist = ''.join([data_dir, train_dist, 'tr_reg_dist_', strnum, '.h5'])
# #         dist.append(load_dist(fn_dist))
# print('loaded all')
    
train_dataset = FAUSTDataset(tr, batch_size)

train_loader = Dataset.DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=0, drop_last=False
                         )

# import model
model = DFM(tr, num_layer)
model = model.float()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08)

# loss
criterion = EuclideanLoss()

for epoch in range(0, max_epoch):
    if epoch % 100 > 0:
        lr *= 0.1
        
    total_loss = 0
    count = 0
    model.train()
    for data in enumerate(train_loader):
#         print(len(data[1]))
#         print(data)
        _, data = data # data came out as (0, [6 different data])
        src_shot, tar_shot, src_eigen, tar_eigen, src_dist, tar_dist = data
        src_shot = src_shot.float()
        tar_shot = tar_shot.float()
        src_eigen = src_eigen.float()
        tar_eigen = tar_eigen.float()
        src_dist = src_dist.float()
        tar_dist = tar_dist.float()
#         print(src_shot.shape)
#         print(tar_shot.shape)
#         print(src_eigen.shape)
#         print(tar_eigen.shape)
#         print(src_dist.shape)
#         print(tar_dist.shape)
#         print(type(src_shot))
#         print(type(tar_shot))
#         print(type(src_eigen))
#         print(type(tar_eigen))
#         print(type(src_dist))
#         print(type(tar_dist))
        
        optimizer.zero_grad()
        # P: soft corr matrix
        P, C = model(src_shot, tar_shot, src_eigen, tar_eigen)
        loss = criterion(P, src_dist, tar_dist)
        total_loss += loss
        count += 4
        loss.backward()
        optimizer.step()
        
    print("[{0}/{1}] Loss: {2}".format(epoch, max_epoch, total_loss/count))
    
    if epoch % 10 == 9:
        torch.save(model.state_dict(), os.path.join('{}.pth'.format(epoch+1)))

[0/50] Loss: 1454.5574951171875
[1/50] Loss: 1559.9486083984375
[2/50] Loss: 1458.0335693359375
[3/50] Loss: 1520.697021484375
[4/50] Loss: 1632.489990234375
[5/50] Loss: 1457.59033203125
[6/50] Loss: 1432.615966796875
[7/50] Loss: 1541.205078125
[8/50] Loss: 1454.70166015625
[9/50] Loss: 1522.05126953125
[10/50] Loss: 1449.7593994140625
[11/50] Loss: 1447.366455078125
[12/50] Loss: 1503.822998046875
[13/50] Loss: 1398.0615234375
[14/50] Loss: 1517.6575927734375
[15/50] Loss: 1441.2149658203125
[16/50] Loss: 1398.0615234375
[17/50] Loss: 1471.8585205078125
[18/50] Loss: 1581.234375
[19/50] Loss: 1440.0350341796875
[20/50] Loss: 1460.638671875
[21/50] Loss: 1467.613525390625
[22/50] Loss: 1503.2357177734375
[23/50] Loss: 1447.366455078125
[24/50] Loss: 1573.1463623046875
[25/50] Loss: 1510.483154296875
[26/50] Loss: 1531.72900390625
[27/50] Loss: 1589.8702392578125
[28/50] Loss: 1531.72900390625
[29/50] Loss: 1577.4384765625
[30/50] Loss: 1474.22265625
[31/50] Loss: 1588.0467529296875
[